In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import subprocess
from pathlib import Path

In [ ]:
MEDIA_PATH = "./media"
URL = "https://aathishsivasubrahmanian.graphy.com"
SESSIONID = "C20D80B62A11008579D7DCE7EDAEA131"
COOKIES = {"SESSIONID":SESSIONID}
SELECTED_COURSE_ID = "6731ae22e5fa980e331decaf"
COURSE_SECTION_NAME = "Introduction to Course"
headers = {
    "accept": "application/json, text/javascript, */*; q=0.01",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "content-type": "application/json; charset=utf-8",
    "priority": "u=1, i",
    "referer": f"{URL}/s/mycourses?type=active&lsb",
    "sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest"
}
media_path = Path(MEDIA_PATH)
unprocessed_media_path = media_path/"unprocessed"
processed_media_path = media_path/"processed"


In [3]:
r = requests.get(f"{URL}/s/mycourses?type=active&lsb",cookies=COOKIES,headers=headers)
apkid = re.search(r'apkId = \"(.*)\", theme9',r.text).group(1)


In [4]:

r = requests.get(f"{URL}/s/courses/{SELECTED_COURSE_ID}/take",cookies=COOKIES,headers=headers)
hlsurl = re.search(r'https:\/\/.*hls.*.min.js',r.text).group(0)
soup = BeautifulSoup(r.content)
mydivs = soup.find("div",{"data-title":COURSE_SECTION_NAME}).find_all("div", {"class": "courseSubItem"})
viddict = {}
for div in mydivs:
    viddict[div.attrs["data-title"]] = div.attrs['data-id']

print(json.dumps(viddict,indent=4))

{
    "SAMPLE_VIDEO": "6731c9518cbb8b3ca42f41f2"
}


In [5]:
for vid in viddict:
    r = requests.get(f"{URL}/s/courses/{SELECTED_COURSE_ID}/videos/{viddict[vid]}/get?_=1699857984568",cookies=COOKIES,headers=headers)
    print(r.text)
    viddict[vid] = r.json()
print(json.dumps(viddict,indent=4))

{"_id":"6731c9518cbb8b3ca42f41f2","spayee:resource":{"spayee:title":"SAMPLE_VIDEO","spayee:courseAssetType":"video","spayee:type":"upload","spayee:underProcess":false,"spayee:completed":true,"spayee:status":"Completed","spayee:gencodePath":"assets/videos/6731ad6d7c5f1d5698393004/2024/11/11/6731c9518cbb8b3ca42f41f2","spayee:previewUrl":"assets/videos/6731ad6d7c5f1d5698393004/2024/11/11/6731c9518cbb8b3ca42f41f2/6731c9518cbb8b3ca42f41f2","spayee:uploadVideoMP4Link":"https://s3-ap-south-1.amazonaws.com/spee-qenc-videos/assets/videos/6731ad6d7c5f1d5698393004/2024/11/11/6731c9518cbb8b3ca42f41f2/6731c9518cbb8b3ca42f41f2","spayee:progress":100,"spayee:coverVersion":1,"spayee:totalTime":0.22466667,"spayee:streamUrl":"https://qcdn.spayee.in/spees/w/o/6731ad6d7c5f1d5698393004/v/6731c9518cbb8b3ca42f41f2/u/6731af219bee1a760d1a6148/t/32715d7f1dee8d878f62c2e6f4a0c81a/p/assets/videos/6731ad6d7c5f1d5698393004/2024/11/11/6731c9518cbb8b3ca42f41f2/index.m3u8"},"watchedVideoDuration":3,"drmPlayerSupport":f

In [6]:
for vid in viddict:
    if not isinstance(viddict[vid],dict):
        continue 

    viddirpath = unprocessed_media_path/vid
    viddirpath.mkdir(exist_ok=True,parents=True)

    print(f"Starting Download : {vid}")
    url = viddict[vid]["spayee:resource"]["spayee:streamUrl"]
    r = requests.get(url,cookies=COOKIES,headers=headers)
    with open(viddirpath/"index.m3u8","wb") as f:
        f.write(r.content)
    NEEDS_SEPARATE_AUDIO = ("hls_audio_.m3u8" in r.text)
    
    r = requests.get(url[:-10]+"hls_500k_.m3u8",cookies=COOKIES,headers=headers)
    with open(viddirpath/"hls_500k_.m3u8","w") as f:
        f.write(r.content.decode("utf-8").replace("k/timestamp","key_video.bin"))
    
    with open(viddirpath/"hls_500k_.m3u8","r") as f:
        with open(viddirpath/"key_video.bin","wb") as f2:
            print(url)
            r = requests.get(url[:-10]+"k/timestamp",cookies=COOKIES,headers=headers)
            timestampreturn = list(bytearray(r.content))
            print(timestampreturn)
            bytestowrite = eval((subprocess.check_output(["node","decrypt_key.js", apkid, str(timestampreturn),r.url])).decode("utf-8").strip())
            print("bytes:",bytestowrite)
            f2.write(bytes(bytestowrite))
        for line in f.readlines():
            if line.startswith("hls_500k"):
                r = requests.get(url[:-10]+line.strip(),cookies=COOKIES,headers=headers)
                with open(viddirpath/line.strip(),"wb") as f2:
                    f2.write(r.content)

    if NEEDS_SEPARATE_AUDIO:
        r = requests.get(url[:-10]+"hls_audio_.m3u8",cookies=COOKIES,headers=headers)
        with open(viddirpath/"hls_audio_.m3u8","w") as f:
            f.write(r.content.decode("utf-8").replace("k/timestamp","key_audio.bin"))
        with open(viddirpath/"hls_audio_.m3u8","r") as f:
            with open(viddirpath/"key_audio.bin","wb") as f2:
                r = requests.get(url[:-10]+"k/timestamp",cookies=COOKIES,headers=headers)
                timestampreturn = list(bytearray(r.content))
                bytestowrite = eval((subprocess.check_output(["node","decrypt_key.js", apkid, str(timestampreturn),r.url])).decode("utf-8").strip())
                f2.write(bytes(bytestowrite))
            for line in f.readlines():
                if line.startswith("hls_audio"):
                    r = requests.get(url[:-10]+line.strip(),cookies=COOKIES)
                    with open(viddirpath/line.strip(),"wb") as f2:
                        f2.write(r.content)
    print(f"Finished : {vid}")
    # break


Starting Download : SAMPLE_VIDEO
https://qcdn.spayee.in/spees/w/o/6731ad6d7c5f1d5698393004/v/6731c9518cbb8b3ca42f41f2/u/6731af219bee1a760d1a6148/t/32715d7f1dee8d878f62c2e6f4a0c81a/p/assets/videos/6731ad6d7c5f1d5698393004/2024/11/11/6731c9518cbb8b3ca42f41f2/index.m3u8
[126, 50, 74, 54, 59, 89, 76, 102, 241, 241, 50, 193, 209, 148, 174, 187, 112, 216, 95, 234, 128, 178, 54, 14, 73, 93, 121, 41, 55, 62, 40, 85, 160, 20, 186, 128, 182, 43, 9, 193, 63, 236, 47, 137, 86, 138, 4, 205]
bytes: [245, 221, 211, 1, 35, 171, 35, 255, 216, 254, 189, 225, 207, 191, 166, 52]
Finished : SAMPLE_VIDEO


In [7]:
# List all folders in the directory
unprocessed_videos = [item for item in unprocessed_media_path.iterdir() if item.is_dir()]
processed_media_path.mkdir(parents=True,exist_ok=True)
for unprocessed_video in unprocessed_videos:
    subprocess.run([
        './ffmpeg-darwin-arm64', 
        '-y', 
        '-allowed_extensions', 'ALL', 
        '-i', f"{unprocessed_video}/index.m3u8", 
        '-c', 'copy', 
        '-vcodec', 'copy', 
        f"{processed_media_path/unprocessed_video.name}.mp4"
    ])



ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2.5)
  configuration: --prefix=/Volumes/tempdisk/sw --extra-cflags=-fno-stack-check --arch=arm64 --cc=/usr/bin/clang --enable-gpl --enable-libvmaf --enable-libbluray --enable-libopenjpeg --enable-libopus --enable-libmp3lame --enable-libx264 --enable-libx265 --enable-libvpx --enable-libwebp --enable-libass --enable-libfreetype --enable-fontconfig --enable-libtheora --enable-libvorbis --enable-libsnappy --enable-libaom --enable-libvidstab --enable-libzimg --enable-libsvtav1 --enable-libkvazaar --enable-version3 --pkg-config-flags=--static --enable-ffplay --enable-postproc --enable-nonfree --enable-neon --enable-runtime-cpudetect --disable-indev=qtkit --disable-indev=x11grab_xcb
  libavutil      58.  2.100 / 58.  2.100
  libavcodec     60.  3.100 / 60.  3.100
  libavformat    60.  3.100 / 60.  3.100
  libavdevice    60.  1.100 / 60.  1.100
  libavfilter     9.  3.100 /